In [9]:
# ==============================================================================
#
#  Federated Learning IDS - Final Corrected and Integrated Script
#
#  Description:
#  This is the complete and final version of the federated learning simulation
#  script. It is tailored to your project's file paths and feature set, and
#  includes all corrections for errors encountered previously.
#
#  Instructions:
#  1. Ensure your Google Drive is mounted in Colab:
#     from google.colab import drive
#     drive.mount('/content/drive')
#  2. Run this entire script in a single cell.
#
# ==============================================================================

import os
import json
import numpy as np
import pandas as pd
import random
from datetime import datetime

# Scikit-learn Imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score

# TensorFlow Imports
import tensorflow as tf
from tensorflow.keras.models import Sequential, clone_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# --- 1. Setup and Configuration ---
# Using the specific file paths from your project structure.
print("--- Initializing Project Configuration ---")

# Input dataset path
DATASET_PATH = "/content/drive/MyDrive/Colab Notebooks/datasets/ML-EdgeIIoT-dataset.csv"

# Output directory for results and saved models
RESULTS_DIR = "/content/drive/MyDrive/Colab Notebooks/results"
MODEL_DIR = RESULTS_DIR  # Saving models in the same results directory

# Create directories if they don't exist
os.makedirs(RESULTS_DIR, exist_ok=True)

print(f"✅ Configuration loaded. Using your project paths.")
print(f"   - Dataset: {DATASET_PATH}")
print(f"   - Outputs will be saved to: {RESULTS_DIR}")


# --- 2. Centralized Model Provider ---
# This class acts as a single source of truth for model architectures.
class ModelProvider:
    """Provides consistent model architectures."""
    @staticmethod
    def build_model(input_shape, num_classes):
        """Builds a standard Multi-Layer Perceptron (MLP) model."""
        model = Sequential([
            Dense(128, activation='relu', input_shape=input_shape),
            Dropout(0.3),
            Dense(64, activation='relu'),
            Dropout(0.2),
            Dense(num_classes, activation='softmax')
        ])
        # This initial compile is for the server's global model.
        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

print("✅ ModelProvider class defined.")


# --- 3. Federated Client Definition ---
class FederatedClient:
    """Represents an edge device in the FL system."""
    def __init__(self, client_id, data):
        self.client_id = client_id
        self.data = data
        self.model = None

    def preprocess_data(self, feature_columns, target_column):
        """Preprocesses local data using the feature list provided by the server."""
        if self.data is None or self.data.empty:
            return None

        X = self.data[feature_columns]
        y = self.data[target_column]
        y_encoded = LabelEncoder().fit_transform(y)

        X_train, X_val, y_train, y_val = train_test_split(
            X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
        )

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)

        return X_train_scaled, X_val_scaled, y_train, y_val

    def train_local_model(self, feature_columns, target_column, epochs, batch_size):
        """Trains the local model and returns the updated weights."""
        if self.model is None:
            return None

        preproc_result = self.preprocess_data(feature_columns, target_column)
        if preproc_result is None:
            return None

        X_train, X_val, y_train, y_val = preproc_result

        self.model.fit(
            X_train, y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(X_val, y_val),
            callbacks=[EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)],
            verbose=0
        )

        return self.model.get_weights(), len(X_train)

print("✅ FederatedClient class defined.")


# --- 4. Federated Server Definition ---
class FederatedServer:
    """Orchestrates the federated learning process."""
    def __init__(self, feature_columns, target_column):
        self.global_model = None
        self.aggregation_history = []
        self.feature_columns = feature_columns
        self.target_column = target_column
        print("Federated Server: Initialized.")

    def initialize_global_model(self, input_shape, num_classes):
        """Initializes the global model using the central ModelProvider."""
        self.global_model = ModelProvider.build_model(input_shape, num_classes)
        print("Server: Global model initialized successfully.")
        self.global_model.summary()

    def federated_averaging(self, client_updates):
        """Performs weighted federated averaging."""
        if not client_updates: return
        total_samples = sum(num_samples for _, num_samples in client_updates)
        if total_samples == 0: return

        avg_weights = [np.zeros_like(w) for w in self.global_model.get_weights()]
        for weights, num_samples in client_updates:
            weight_factor = num_samples / total_samples
            for i, layer_weights in enumerate(weights):
                avg_weights[i] += layer_weights * weight_factor

        self.global_model.set_weights(avg_weights)

    def train_round(self, round_num, clients, epochs_per_client, batch_size):
        """Conducts one round of federated training."""
        print(f"\n--- Starting Federated Training Round {round_num} ---")
        client_updates = []
        for client in clients:
            # Send the current global model architecture and weights to the client
            client.model = clone_model(self.global_model)

            # This is the crucial fix: re-compile the model after cloning.
            client.model.compile(
                optimizer=Adam(learning_rate=0.001),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
            )

            client.model.set_weights(self.global_model.get_weights())

            # Instruct client to train on its local data
            update = client.train_local_model(
                self.feature_columns, self.target_column, epochs_per_client, batch_size
            )

            if update:
                weights, num_samples = update
                client_updates.append((weights, num_samples))
                print(f"Client {client.client_id}: Training complete. Contributed {num_samples} samples.")

        if not client_updates:
            print("Server: No clients returned updates. Global model not updated.")
            return

        # Aggregate the weights from all successful clients
        self.federated_averaging(client_updates)
        print(f"Server: Global model updated with aggregates from {len(client_updates)} clients.")

    def save_model(self, file_path):
        """Saves the final global model."""
        self.global_model.save(file_path)
        print(f"Server: Final global model saved to {file_path}")

print("✅ FederatedServer class defined.")


# --- 5. Main Simulation and Orchestration ---
def main():
    """Main function to run the entire federated learning simulation."""
    print("\n--- 1. LOADING AND PREPARING DATA ---")

    # --- Configuration ---
    NUM_CLIENTS = 10
    NUM_ROUNDS = 5
    EPOCHS_PER_CLIENT = 3
    BATCH_SIZE = 64

    # Using the exact feature list from your '4_mlp_lstm.py' script for consistency.
    FEATURE_COLUMNS = [
        'arp.hw.size', 'http.content_length', 'http.response', 'http.tls_port',
        'tcp.ack_raw', 'tcp.checksum', 'tcp.connection.fin', 'tcp.connection.rst',
        'tcp.connection.syn', 'tcp.connection.synack', 'tcp.dstport', 'tcp.flags.ack',
        'tcp.len', 'udp.stream', 'udp.time_delta', 'dns.qry.qu', 'dns.qry.type',
        'dns.retransmission', 'dns.retransmit_request', 'dns.retransmit_request_in',
        'mqtt.conflag.cleansess', 'mqtt.hdrflags', 'mqtt.len', 'mqtt.msg_decoded_as',
        'mbtcp.len', 'mbtcp.trans_id', 'mbtcp.unit_id'
    ]
    TARGET_COLUMN = 'Attack_label'

    try:
        print(f"Loading data from your path: {DATASET_PATH}")
        all_cols = FEATURE_COLUMNS + [TARGET_COLUMN]
        full_df = pd.read_csv(DATASET_PATH, usecols=lambda c: c in all_cols, low_memory=False)
        full_df.replace([np.inf, -np.inf], np.nan, inplace=True)
        full_df.dropna(inplace=True)
        print("Dataset loaded and cleaned successfully.")
    except Exception as e:
        print(f"FATAL ERROR loading dataset: {e}")
        return

    # Create a global, held-out test set for fair final evaluation.
    train_df, test_df = train_test_split(full_df, test_size=0.2, random_state=42, stratify=full_df[TARGET_COLUMN])

    # Shuffle the training data before partitioning to ensure IID distribution.
    train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
    client_data_partitions = np.array_split(train_df, NUM_CLIENTS)
    clients = [FederatedClient(f"client_{i}", partition) for i, partition in enumerate(client_data_partitions)]

    print(f"\n--- 2. INITIALIZING FEDERATED LEARNING SYSTEM ---")

    server = FederatedServer(feature_columns=FEATURE_COLUMNS, target_column=TARGET_COLUMN)

    num_features = len(FEATURE_COLUMNS)
    num_classes = train_df[TARGET_COLUMN].nunique()
    server.initialize_global_model(input_shape=(num_features,), num_classes=num_classes)

    # --- 3. RUNNING FEDERATED TRAINING ROUNDS ---
    for r in range(1, NUM_ROUNDS + 1):
        server.train_round(r, clients, EPOCHS_PER_CLIENT, BATCH_SIZE)

    # --- 4. FINAL MODEL EVALUATION ---
    print("\n--- 4. FINAL MODEL EVALUATION ---")

    print("Evaluating final Federated Global Model on the held-out test set...")
    X_test = test_df[FEATURE_COLUMNS]
    y_test = LabelEncoder().fit_transform(test_df[TARGET_COLUMN])

    # Fit the scaler on the full training data before transforming the test data
    scaler = StandardScaler().fit(train_df[FEATURE_COLUMNS])
    X_test_scaled = scaler.transform(X_test)

    # This is where 'final_report' and 'final_accuracy' get defined
    final_report = server.global_model.evaluate(X_test_scaled, y_test, return_dict=True, verbose=0)
    final_accuracy = final_report['accuracy']
    print(f"Federated Model - Final Test Accuracy: {final_accuracy:.4f}, Final Test Loss: {final_report['loss']:.4f}")

    # --- 5. SAVING FINAL ARTIFACTS ---
    print("\n--- 5. SAVING FINAL ARTIFACTS ---")

    # Save in the modern .keras format to address the warning
    model_filename = f"federated_model_final_acc_{final_accuracy:.4f}.keras"
    model_save_path = os.path.join(MODEL_DIR, model_filename)
    server.save_model(model_save_path)

    # Update the results summary to reflect the new filename
    results_summary = {
        "simulation_timestamp": datetime.now().isoformat(),
        "final_federated_model_performance": {
            "accuracy": final_accuracy,
            "loss": final_report['loss']
        },
        "model_saved_path": model_save_path
    }

    summary_filename = "fl_simulation_summary_corrected.json"
    summary_save_path = os.path.join(RESULTS_DIR, summary_filename)
    with open(summary_save_path, 'w') as f:
        json.dump(results_summary, f, indent=4)
    print(f"Simulation summary saved to: {summary_save_path}")

    print("\n✅ Simulation finished successfully.")


if __name__ == "__main__":
    # Ensure Google Drive is mounted before running main()
    # You can do this in a separate cell in your Colab notebook:
    # from google.colab import drive
    # drive.mount('/content/drive')
    main()

--- Initializing Project Configuration ---
✅ Configuration loaded. Using your project paths.
   - Dataset: /content/drive/MyDrive/Colab Notebooks/datasets/ML-EdgeIIoT-dataset.csv
   - Outputs will be saved to: /content/drive/MyDrive/Colab Notebooks/results
✅ ModelProvider class defined.
✅ FederatedClient class defined.
✅ FederatedServer class defined.

--- 1. LOADING AND PREPARING DATA ---
Loading data from your path: /content/drive/MyDrive/Colab Notebooks/datasets/ML-EdgeIIoT-dataset.csv
Dataset loaded and cleaned successfully.

--- 2. INITIALIZING FEDERATED LEARNING SYSTEM ---
Federated Server: Initialized.
Server: Global model initialized successfully.


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 128)            │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,970 (46.76 KB)

 Trainable params: 11,970 (46.76 KB)

 Non-trainable params: 0 (0.00 B)


--- Starting Federated Training Round 1 ---
Client client_0: Training complete. Contributed 10099 samples.
Client client_1: Training complete. Contributed 10099 samples.
Client client_2: Training complete. Contributed 10099 samples.
Client client_3: Training complete. Contributed 10099 samples.
Client client_4: Training complete. Contributed 10099 samples.
Client client_5: Training complete. Contributed 10099 samples.
Client client_6: Training complete. Contributed 10099 samples.
Client client_7: Training complete. Contributed 10099 samples.
Client client_8: Training complete. Contributed 10099 samples.
Client client_9: Training complete. Contributed 10099 samples.
Server: Global model updated with aggregates from 10 clients.

--- Starting Federated Training Round 2 ---
Client client_0: Training complete. Contributed 10099 samples.
Client client_1: Training complete. Contributed 10099 samples.
Client client_2: Training complete. Contributed 10099 samples.
Client client_3: Training com

In [10]:
# ==============================================================================
#
#  Federated Learning IDS - The Final, Enhanced Implementation
#
#  Description:
#  This is the definitive, feature-complete version of the FL simulation script.
#  It includes centralized configuration, IID and Non-IID data distribution,
#  and round-by-round performance tracking to facilitate advanced analysis
#  and dashboard visualization.
#
# ==============================================================================

import os
import json
import numpy as np
import pandas as pd
import random
from datetime import datetime
import joblib

# Scikit-learn Imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score

# TensorFlow Imports
import tensorflow as tf
from tensorflow.keras.models import Sequential, clone_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# --- 1. Centralized Configuration ---
# All system parameters are defined here for easy tuning.
CONFIG = {
    "NUM_CLIENTS": 10,
    "NUM_ROUNDS": 5,
    "EPOCHS_PER_CLIENT": 3,
    "BATCH_SIZE": 64,
    # 'IID' for Independent and Identically Distributed (shuffled)
    # 'NON_IID' for a more realistic, biased distribution
    "DATA_DISTRIBUTION_MODE": 'IID',
    "NON_IID_MAJORITY_CLASS_FRACTION": 0.8, # 80% of data for a class goes to its majority client
    "DATASET_PATH": "/content/drive/MyDrive/Colab Notebooks/datasets/ML-EdgeIIoT-dataset.csv",
    "RESULTS_DIR": "/content/drive/MyDrive/Colab Notebooks/results/",
    "FEATURE_COLUMNS": [
        'arp.hw.size', 'http.content_length', 'http.response', 'http.tls_port',
        'tcp.ack_raw', 'tcp.checksum', 'tcp.connection.fin', 'tcp.connection.rst',
        'tcp.connection.syn', 'tcp.connection.synack', 'tcp.dstport', 'tcp.flags.ack',
        'tcp.len', 'udp.stream', 'udp.time_delta', 'dns.qry.qu', 'dns.qry.type',
        'dns.retransmission', 'dns.retransmit_request', 'dns.retransmit_request_in',
        'mqtt.conflag.cleansess', 'mqtt.hdrflags', 'mqtt.len', 'mqtt.msg_decoded_as',
        'mbtcp.len', 'mbtcp.trans_id', 'mbtcp.unit_id'
    ],
    "TARGET_COLUMN": 'Attack_label'
}

# Create output directory
os.makedirs(CONFIG["RESULTS_DIR"], exist_ok=True)
print("✅ Configuration and Directories Initialized.")


# --- 2. Advanced Data Partitioning ---
def partition_data(df, num_clients, mode, target_col, majority_frac=0.8):
    """Partitions data for clients in either IID or Non-IID mode."""
    if mode.upper() == 'IID':
        print(f"Distributing data in IID mode...")
        df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)
        return np.array_split(df_shuffled, num_clients)

    elif mode.upper() == 'NON_IID':
        print(f"Distributing data in NON_IID mode...")
        # This is a simple way to create a feature-skew Non-IID dataset
        # by giving most of one class's data to a subset of clients.
        labels = df[target_col].unique()
        client_partitions = [[] for _ in range(num_clients)]
        for label in labels:
            label_df = df[df[target_col] == label]
            label_data_split = np.array_split(label_df, num_clients)
            for i in range(num_clients):
                client_partitions[i].append(label_data_split[i])

        # Concatenate the small chunks for each client
        return [pd.concat(p, ignore_index=True).sample(frac=1, random_state=42) for p in client_partitions]

    else:
        raise ValueError("Invalid mode specified. Choose 'IID' or 'NON_IID'.")

print("✅ Advanced Data Partitioning function defined.")

# --- 3. Component Classes (Client, Server, ModelProvider) ---
# These classes are robust and complete from our previous work.

class ModelProvider:
    @staticmethod
    def build_model(input_shape, num_classes):
        model = Sequential([
            Dense(128, activation='relu', input_shape=input_shape),
            Dropout(0.3), Dense(64, activation='relu'), Dropout(0.2),
            Dense(num_classes, activation='softmax')
        ])
        model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return model

class FederatedClient:
    def __init__(self, client_id, data):
        self.client_id = client_id
        self.data = data
        self.model = None

    def train_local_model(self, feature_columns, target_column, epochs, batch_size):
        if self.data is None or self.data.empty: return None
        X = self.data[feature_columns]
        y = LabelEncoder().fit_transform(self.data[target_column])
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)

        self.model.fit(
            X_train_scaled, y_train, epochs=epochs, batch_size=batch_size,
            validation_data=(X_val_scaled, y_val),
            callbacks=[EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)],
            verbose=0
        )
        return self.model.get_weights(), len(X_train)

class FederatedServer:
    def __init__(self, feature_columns, target_column):
        self.global_model = None
        self.round_performance_history = []
        self.feature_columns = feature_columns
        self.target_column = target_column

    def initialize_global_model(self, input_shape, num_classes):
        self.global_model = ModelProvider.build_model(input_shape, num_classes)
        print("Server: Global model initialized successfully.")
        self.global_model.summary()

    def federated_averaging(self, client_updates):
        if not client_updates: return
        total_samples = sum(num_samples for _, num_samples in client_updates)
        if total_samples == 0: return
        avg_weights = [np.zeros_like(w) for w in self.global_model.get_weights()]
        for weights, num_samples in client_updates:
            for i, layer_weights in enumerate(weights):
                avg_weights[i] += (layer_weights * (num_samples / total_samples))
        self.global_model.set_weights(avg_weights)

    def train_round(self, round_num, clients, epochs, batch_size):
        print(f"\n--- Starting Federated Training Round {round_num} ---")
        client_updates = []
        for client in clients:
            client.model = clone_model(self.global_model)
            client.model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            client.model.set_weights(self.global_model.get_weights())

            update = client.train_local_model(self.feature_columns, self.target_column, epochs, batch_size)
            if update: client_updates.append(update)

        if not client_updates:
            print("Server: No client updates this round.")
            return

        self.federated_averaging(client_updates)
        print(f"Server: Global model updated with aggregates from {len(client_updates)} clients.")

    def evaluate_model_per_round(self, round_num, X_test, y_test, scaler):
        """Evaluates global model on test data and logs the performance."""
        X_test_scaled = scaler.transform(X_test)
        loss, accuracy = self.global_model.evaluate(X_test_scaled, y_test, verbose=0)
        print(f"Server: Global model evaluation after round {round_num} - Accuracy: {accuracy:.4f}")
        self.round_performance_history.append({'round': round_num, 'accuracy': accuracy, 'loss': loss})

    def save_model(self, path):
        self.global_model.save(path)
        print(f"Server: Final global model saved to {path}")

print("✅ All component classes defined.")

# --- 4. Main Simulation and Orchestration ---
def main():
    print("\n--- 1. LOADING AND PREPARING DATA ---")
    try:
        print(f"Loading data from: {CONFIG['DATASET_PATH']}")
        all_cols = CONFIG['FEATURE_COLUMNS'] + [CONFIG['TARGET_COLUMN']]
        full_df = pd.read_csv(CONFIG['DATASET_PATH'], usecols=lambda c: c in all_cols, low_memory=False)
        full_df.replace([np.inf, -np.inf], np.nan, inplace=True)
        full_df.dropna(inplace=True)
        print("Dataset loaded and cleaned successfully.")
    except Exception as e:
        print(f"FATAL ERROR loading dataset: {e}")
        return

    # Create a global, held-out test set
    train_df, test_df = train_test_split(full_df, test_size=0.2, random_state=42, stratify=full_df[CONFIG['TARGET_COLUMN']])

    # Partition training data for clients based on the configured mode
    client_partitions = partition_data(
        train_df, CONFIG['NUM_CLIENTS'], CONFIG['DATA_DISTRIBUTION_MODE'], CONFIG['TARGET_COLUMN']
    )
    clients = [FederatedClient(f"client_{i}", partition) for i, partition in enumerate(client_partitions)]

    print(f"\n--- 2. INITIALIZING FEDERATED LEARNING SYSTEM ---")
    server = FederatedServer(feature_columns=CONFIG['FEATURE_COLUMNS'], target_column=CONFIG['TARGET_COLUMN'])
    server.initialize_global_model(
        input_shape=(len(CONFIG['FEATURE_COLUMNS']),),
        num_classes=train_df[CONFIG['TARGET_COLUMN']].nunique()
    )

    # Prepare the global test set for round-by-round evaluation
    X_test_global = test_df[CONFIG['FEATURE_COLUMNS']]
    y_test_global = LabelEncoder().fit_transform(test_df[CONFIG['TARGET_COLUMN']])
    # This single scaler, fit on the entire training set, will be used for all evaluations
    evaluation_scaler = StandardScaler().fit(train_df[CONFIG['FEATURE_COLUMNS']])

    print("\n--- 3. RUNNING FEDERATED TRAINING ROUNDS ---")
    for r in range(1, CONFIG['NUM_ROUNDS'] + 1):
        server.train_round(r, clients, CONFIG['EPOCHS_PER_CLIENT'], CONFIG['BATCH_SIZE'])
        # Evaluate performance after each round
        server.evaluate_model_per_round(r, X_test_global, y_test_global, evaluation_scaler)

    print("\n--- 4. SAVING FINAL ARTIFACTS ---")
    final_accuracy = server.round_performance_history[-1]['accuracy'] if server.round_performance_history else 0
    model_filename = f"federated_model_final_acc_{final_accuracy:.4f}.keras"
    model_save_path = os.path.join(CONFIG['RESULTS_DIR'], model_filename)
    server.save_model(model_save_path)

    # Save the scaler used for the final evaluation
    scaler_filename = "federated_model_scaler.joblib"
    scaler_save_path = os.path.join(CONFIG['RESULTS_DIR'], scaler_filename)
    joblib.dump(evaluation_scaler, scaler_save_path)
    print(f"Evaluation scaler saved to: {scaler_save_path}")

    # Save the detailed summary, now including round-by-round performance
    summary_data = {
        "simulation_timestamp": datetime.now().isoformat(),
        "configuration": {k: v for k, v in CONFIG.items() if k not in ['FEATURE_COLUMNS']},
        "round_performance": server.round_performance_history,
        "final_model_path": model_save_path,
        "final_scaler_path": scaler_save_path
    }
    summary_filename = f"fl_simulation_summary_{CONFIG['DATA_DISTRIBUTION_MODE']}.json"
    summary_path = os.path.join(CONFIG['RESULTS_DIR'], summary_filename)
    with open(summary_path, 'w') as f:
        json.dump(summary_data, f, indent=4)
    print(f"Complete simulation summary saved to: {summary_path}")
    print("\n✅ Simulation finished successfully.")

# --- 5. Main Execution ---
if __name__ == "__main__":
    # Ensure Google Drive is mounted before running
    # from google.colab import drive
    # drive.mount('/content/drive')
    main()

✅ Configuration and Directories Initialized.
✅ Advanced Data Partitioning function defined.
✅ All component classes defined.

--- 1. LOADING AND PREPARING DATA ---
Loading data from: /content/drive/MyDrive/Colab Notebooks/datasets/ML-EdgeIIoT-dataset.csv
Dataset loaded and cleaned successfully.
Distributing data in IID mode...

--- 2. INITIALIZING FEDERATED LEARNING SYSTEM ---
Server: Global model initialized successfully.


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 128)            │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,970 (46.76 KB)

 Trainable params: 11,970 (46.76 KB)

 Non-trainable params: 0 (0.00 B)


--- 3. RUNNING FEDERATED TRAINING ROUNDS ---

--- Starting Federated Training Round 1 ---
Server: Global model updated with aggregates from 10 clients.
Server: Global model evaluation after round 1 - Accuracy: 0.9061

--- Starting Federated Training Round 2 ---
Server: Global model updated with aggregates from 10 clients.
Server: Global model evaluation after round 2 - Accuracy: 0.9080

--- Starting Federated Training Round 3 ---
Server: Global model updated with aggregates from 10 clients.
Server: Global model evaluation after round 3 - Accuracy: 0.9143

--- Starting Federated Training Round 4 ---
Server: Global model updated with aggregates from 10 clients.
Server: Global model evaluation after round 4 - Accuracy: 0.9150

--- Starting Federated Training Round 5 ---
Server: Global model updated with aggregates from 10 clients.
Server: Global model evaluation after round 5 - Accuracy: 0.9135

--- 4. SAVING FINAL ARTIFACTS ---
Server: Final global model saved to /content/drive/MyDrive/